## Latent Dirichlet Allocation application for topic modeling ##

LDA is used to classify text in a document to a particular topic. It builds a topic per document model and words per topic model, modeled as Dirichlet distributions. 

* Each document is modeled as a multinomial distribution of topics and each topic is modeled as a multinomial distribution of words.
* LDA assumes that the every chunk of text we feed into it will contain words that are somehow related. Therefore choosing the right corpus of data is crucial. 
* It also assumes documents are produced from a mixture of topics. Those topics then generate words based on their probability distribution. 

This is an **unsupervised learning** task

The dataset is a list of over one million news headlines published over a period of 15 years. 

In [1]:

import pandas as pd
data = pd.read_csv('abcnews-date-text.csv', error_bad_lines=False);
# We only need the Headlines text column from the data
data_text = data[:300000][['headline_text']];
data_text['index'] = data_text.index

documents = data_text

Looking into the dataset:

In [2]:
print(len(documents))

300000


In [3]:
documents[:5]

,headline_text,index
0,aba decides against community broadcasting lic...,0
1,act fire witnesses must be aware of defamation,1
2,a g calls for infrastructure protection summit,2
3,air nz staff in aust strike for pay rise,3
4,air nz strike to affect australian travellers,4


## Data Preprocessing ##


* **Tokenization**: Split the text into sentences and the sentences into words. Lowercase the words and remove punctuation.
* Words that have fewer than 3 characters are removed.
* All **stopwords** are removed.
* Words are **lemmatized** - words in third person are changed to first person and verbs in past and future tenses are changed into present.
* Words are **stemmed** - words are reduced to their root form.


In [4]:
'''
Loading Gensim and nltk libraries
'''
# pip install gensim
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(400)

In [5]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

### Lemmatizer Example
Before preprocessing our dataset, let's first look at an lemmatizing example. What would be the output if we lemmatized the word 'went':

In [6]:
print(WordNetLemmatizer().lemmatize('went', pos = 'v')) # past tense to present tense

go


### Stemming

In [7]:
stemmer = SnowballStemmer("english")
original_words = ['caresses', 'flies', 'dies', 'mules', 'denied','died', 'agreed', 'owned', 
           'humbled', 'sized','meeting', 'stating', 'siezing', 'itemization','sensational', 
           'traditional', 'reference', 'colonizer','plotted']
singles = [stemmer.stem(plural) for plural in original_words]

pd.DataFrame(data={'original word':original_words, 'stemmed':singles })

,original word,stemmed
0,caresses,caress
1,flies,fli
2,dies,die
3,mules,mule
4,denied,deni
5,died,die
6,agreed,agre
7,owned,own
8,humbled,humbl
9,sized,size


In [8]:
'''
Function to perform the pre processing steps on the entire dataset
'''
def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))

# Tokenize and lemmatize
def preprocess(text):
    result=[]
    for token in gensim.utils.simple_preprocess(text) :
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            # Apply lemmatize_stemming() on the token, then add to the results list
            result.append( lemmatize_stemming( token ) )
    return result

In [9]:
'''
Preview a document after preprocessing
'''
document_num = 4310
doc_sample = documents[documents['index'] == document_num].values[0][0]

print("Original document: ")
words = []
for word in doc_sample.split(' '):
    words.append(word)
print(words)
print("\n\nTokenized and lemmatized document: ")
print(preprocess(doc_sample))

Original document: 
['rain', 'helps', 'dampen', 'bushfires']


Tokenized and lemmatized document: 
['rain', 'help', 'dampen', 'bushfir']


Time to preprocess all the news headlines using the [map](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.Series.map.html) function from pandas to apply `preprocess()` to the `headline_text` column


In [10]:
# from pandas import map
#note: this is taking a few mins
docs = documents['headline_text']
processed_docs = docs.map( preprocess)

In [11]:
'''
Preview 'processed_docs'
'''
processed_docs[:10]

0            [decid, communiti, broadcast, licenc]
1                               [wit, awar, defam]
2           [call, infrastructur, protect, summit]
3                      [staff, aust, strike, rise]
4             [strike, affect, australian, travel]
5               [ambiti, olsson, win, tripl, jump]
6           [antic, delight, record, break, barca]
7    [aussi, qualifi, stosur, wast, memphi, match]
8            [aust, address, secur, council, iraq]
9                         [australia, lock, timet]
Name: headline_text, dtype: object

## Bag of words on the dataset

Create a dictionary from 'processed_docs' containing the number of times a word appears in the training set. `processed_docs` is passed to [`gensim.corpora.Dictionary()`](https://radimrehurek.com/gensim/corpora/dictionary.html).

In [12]:
'''
Create a dictionary from 'processed_docs' containing the number of times a word appears 
in the training set using gensim.corpora.Dictionary
'''
dictionary = gensim.corpora.Dictionary(processed_docs)

In [13]:
'''
Checking dictionary created
'''
count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break

0 broadcast
1 communiti
2 decid
3 licenc
4 awar
5 defam
6 wit
7 call
8 infrastructur
9 protect
10 summit


** Gensim filter_extremes **

[`filter_extremes(no_below=5, no_above=0.5, keep_n=100000)`](https://radimrehurek.com/gensim/corpora/dictionary.html#gensim.corpora.dictionary.Dictionary.filter_extremes)

Filtering out tokens that appear in

* less than no_below documents (absolute number) or
* more than no_above documents (fraction of total corpus size, not absolute number).
* after (1) and (2), keep only the first keep_n most frequent tokens (or keep all if None).

In [14]:
'''
Remove very rare and very common words:

- words appearing less than 15 times
- words appearing in more than 10% of all documents
'''
dictionary.filter_extremes(no_below=5, no_above=0.5, keep_n=100000)

** Gensim doc2bow **

[`doc2bow(document)`](https://radimrehurek.com/gensim/corpora/dictionary.html#gensim.corpora.dictionary.Dictionary.doc2bow)

* Converting document (a list of words) into the bag-of-words format = list of (token_id, token_count) 2-tuples. Each word is assumed to be a tokenized and normalized string (either unicode or utf8-encoded). No further preprocessing is done on the words in document; apply tokenization, stemming etc. before calling this method.

In [15]:
'''
Creating the Bag-of-words model for each document i.e for each document we create a dictionary reporting how many
words and how many times those words appear. 
'''
# TODO
bow_corpus = [dictionary.doc2bow( doc ) for doc in processed_docs]
# for doc in processed_docs:
#     bow_corpus.append( dictionary.doc2bow( doc ) )

In [16]:
'''
Checking Bag of Words corpus for sample document --> (token_id, token_count)
'''
bow_corpus[document_num]

[(76, 1), (113, 1), (482, 1), (4016, 1)]

In [17]:
'''
Preview BOW for sample preprocessed document
'''
# Here document_num is document number 4310 which was checked earlier in the notebook 
bow_doc_4310 = bow_corpus[document_num]

for i in range(len(bow_doc_4310)):
    print("Word {} (\"{}\") appears {} time.".format(bow_doc_4310[i][0], 
                                                     dictionary[bow_doc_4310[i][0]], 
                                                     bow_doc_4310[i][1]))

Word 76 ("bushfir") appears 1 time.
Word 113 ("help") appears 1 time.
Word 482 ("rain") appears 1 time.
Word 4016 ("dampen") appears 1 time.


## TF-IDF on document set ##

While performing TF-IDF on the corpus is not necessary for LDA implemention using the gensim model, it is useful. TF-IDF expects a bag-of-words (integer values) training corpus during initialization. During transformation, it will take a vector and return another vector of the same dimensionality.

*The author of Gensim dictates the standard procedure for LDA to be using the Bag of Words model.*

** TF-IDF stands for "Term Frequency, Inverse Document Frequency".**

* It is a way to score the importance of words (or "terms") in a document based on how frequently they appear across multiple documents.
* If a word appears frequently in a document, it's important. Give the word a high score. But if a word appears in many documents, it's not a unique identifier. Give the word a low score.
* Therefore, common words like "the" and "for", which appear in many documents, will be scaled down. Words that appear frequently in a single document will be scaled up.

In other words:

* TF(w) = `(Number of times term w appears in a document) / (Total number of terms in the document)`.
* IDF(w) = `log_e(Total number of documents / Number of documents with term w in it)`.

In [18]:
'''
Create tf-idf model object using models.TfidfModel on 'bow_corpus' and save it
'''
from gensim import corpora, models

tfidf = models.TfidfModel(bow_corpus)

In [19]:
'''
Apply transformation to the entire corpus and call it 'corpus_tfidf'
'''

corpus_tfidf = [ tfidf[ doc ] for doc in bow_corpus ]

In [20]:
'''
Preview TF-IDF scores for our first document --> --> (token_id, tfidf score)
'''
from pprint import pprint
for doc in corpus_tfidf:
    pprint(doc)
    break

[(0, 0.5959813347777092),
 (1, 0.39204529549491984),
 (2, 0.48531419274988147),
 (3, 0.50554610985785686)]


## LDA using Bag of Words ##

Trying 10 topics in the document corpus.

** Running LDA using all CPU cores to parallelize and speed up model training.**

Some of the parameters to be tweaked are:

* **num_topics** is the number of requested latent topics to be extracted from the training corpus.
* **id2word** is a mapping from word ids (integers) to words (strings). It is used to determine the vocabulary size, as well as for debugging and topic printing.
* **workers** is the number of extra processes to use for parallelization. Uses all available cores by default.
* **alpha** and **eta** are hyperparameters that affect sparsity of the document-topic (theta) and topic-word (lambda) distributions. We will let these be the default values for now(default value is `1/num_topics`)
    - Alpha is the per document topic distribution.
        * High alpha: Every document has a mixture of all topics(documents appear similar to each other).
        * Low alpha: Every document has a mixture of very few topics

    - Eta is the per topic word distribution.
        * High eta: Each topic has a mixture of most words(topics appear similar to each other).
        * Low eta: Each topic has a mixture of few words.

* ** passes ** is the number of training passes through the corpus. For  example, if the training corpus has 50,000 documents, chunksize is  10,000, passes is 2, then online training is done in 10 updates: 
    * `#1 documents 0-9,999 `
    * `#2 documents 10,000-19,999 `
    * `#3 documents 20,000-29,999 `
    * `#4 documents 30,000-39,999 `
    * `#5 documents 40,000-49,999 `
    * `#6 documents 0-9,999 `
    * `#7 documents 10,000-19,999 `
    * `#8 documents 20,000-29,999 `
    * `#9 documents 30,000-39,999 `
    * `#10 documents 40,000-49,999` 

In [21]:
# LDA mono-core -- fallback code in case LdaMulticore throws an error on your machine
# lda_model = gensim.models.LdaModel(bow_corpus, 
#                                    num_topics = 10, 
#                                    id2word = dictionary,                                    
#                                    passes = 50)

# LDA multicore 
'''
Train lda model using gensim.models.LdaMulticore and save it
'''

lda_model = gensim.models.LdaMulticore(bow_corpus, 
                                       id2word=dictionary, 
                                       num_topics=10,
#                                        workers=3, #if running on my local quadcore machine
                                       passes=2)

In [23]:
'''
For each topic, explore the words occuring in that topic and its relative weight
'''
for idx, topic in lda_model.print_topics(-1):
    print("Topic: {} \nWords: {}".format(topic, idx ))
    print("\n")

Topic: 0.037*"govt" + 0.027*"urg" + 0.023*"closer" + 0.022*"labor" + 0.022*"claim" + 0.020*"opposit" + 0.018*"elect" + 0.016*"reject" + 0.014*"say" + 0.011*"threat" 
Words: 0


Topic: 0.021*"nation" + 0.021*"hospit" + 0.019*"health" + 0.015*"mayor" + 0.014*"fight" + 0.014*"water" + 0.013*"work" + 0.013*"hick" + 0.013*"rudd" + 0.012*"say" 
Words: 1


Topic: 0.029*"crash" + 0.029*"kill" + 0.029*"iraq" + 0.018*"die" + 0.017*"blaze" + 0.016*"troop" + 0.012*"firefight" + 0.012*"law" + 0.012*"blame" + 0.012*"target" 
Words: 2


Topic: 0.015*"indigen" + 0.015*"talk" + 0.015*"warn" + 0.015*"coast" + 0.014*"nuclear" + 0.013*"rain" + 0.012*"timor" + 0.011*"storm" + 0.011*"open" + 0.011*"north" 
Words: 3


Topic: 0.040*"plan" + 0.040*"council" + 0.036*"water" + 0.021*"govt" + 0.018*"urg" + 0.018*"chang" + 0.015*"power" + 0.014*"back" + 0.013*"resid" + 0.012*"group" 
Words: 4


Topic: 0.022*"forc" + 0.019*"concern" + 0.017*"sydney" + 0.017*"howard" + 0.017*"price" + 0.016*"high" + 0.015*"market" +

### Classification of the topics ###

Using the words in each topic and their corresponding weights, a rough identification of categories as I see it , would be as follows, but I see some ambiguity with just 2 passes

* 0: politics
* 1: British NHS
* 2: International/military
* 3: Support and opposition of nuclear energy
* 4: Water/insfrastructure
* 5: Australia
* 6: Cricket/sports
* 7: Law and order
* 8: Education/gov. spending
* 9: Crime

## Running LDA using TF-IDF ##

In [27]:
'''
Define lda model using corpus_tfidf, using gensim.models.LdaMulticore()
'''

lda_model_tfidf = gensim.models.LdaMulticore(bow_corpus, 
                                       id2word=dictionary, 
                                       num_topics=10,
#                                        workers=3, #if running on my previous local quadcore machine
                                       passes=5)

In [28]:
'''
For each topic, we will explore the words occuring in that topic and its relative weight
'''
for idx, topic in lda_model_tfidf.print_topics(-1):
    print("Topic: {} Word: {}".format(idx, topic))
    print("\n")

Topic: 0 Word: 0.019*"rise" + 0.018*"year" + 0.017*"green" + 0.016*"road" + 0.015*"price" + 0.013*"drought" + 0.012*"region" + 0.011*"seek" + 0.011*"farm" + 0.010*"farmer"


Topic: 1 Word: 0.057*"polic" + 0.028*"investig" + 0.019*"death" + 0.017*"support" + 0.014*"victim" + 0.013*"trial" + 0.013*"begin" + 0.012*"bodi" + 0.012*"prompt" + 0.011*"appeal"


Topic: 2 Word: 0.029*"miss" + 0.020*"search" + 0.017*"break" + 0.016*"look" + 0.015*"polic" + 0.013*"storm" + 0.013*"driver" + 0.012*"launch" + 0.011*"damag" + 0.011*"research"


Topic: 3 Word: 0.041*"charg" + 0.035*"court" + 0.033*"face" + 0.029*"polic" + 0.024*"jail" + 0.022*"accus" + 0.020*"drug" + 0.016*"murder" + 0.015*"probe" + 0.014*"teen"


Topic: 4 Word: 0.025*"open" + 0.023*"lead" + 0.016*"win" + 0.015*"england" + 0.015*"feder" + 0.014*"world" + 0.013*"final" + 0.012*"aussi" + 0.010*"victori" + 0.010*"tiger"


Topic: 5 Word: 0.036*"plan" + 0.023*"water" + 0.017*"council" + 0.017*"closer" + 0.015*"govt" + 0.014*"consid" + 0.014

### Classification of the topics ###

As we can see, when using tf-idf, heavier weights are given to words that are not as frequent which results in nouns being factored in. That makes it harder to figure out the categories as nouns can be hard to categorize. This goes to show that the models we apply depend on the type of corpus of text we are dealing with. 

Using the words in each topic and their corresponding weights, I now see these categories:

* 0: Agriculture
* 1: Law/Crime 
* 2: Legal
* 3: Crime
* 4: Sports 
* 5: City/Planning
* 6: Geopolitics
* 7: Local politics
* 8: Labor
* 9: Australasia

## Performance evaluation by classifying sample document using LDA Bag of Words model##

Check to see where our test document would be classified. 

In [25]:
'''
Text of sample document 4310
'''
processed_docs[4310]

['rain', 'help', 'dampen', 'bushfir']

In [31]:
lda_model[bow_corpus[document_num]]

[(0, 0.019999998),
 (1, 0.020003865),
 (2, 0.019999998),
 (3, 0.40239349),
 (4, 0.020006772),
 (5, 0.23758331),
 (6, 0.019999998),
 (7, 0.22000006),
 (8, 0.02001247),
 (9, 0.020000005)]

In [24]:
'''
Check which topic our test document belongs to using the LDA Bag of Words model.
'''
document_num = 4310
# test document is document number 4310

for index, score in sorted(lda_model[bow_corpus[document_num]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model.print_topic(index, 10)))


Score: 0.40243610739707947	 
Topic: 0.015*"indigen" + 0.015*"talk" + 0.015*"warn" + 0.015*"coast" + 0.014*"nuclear" + 0.013*"rain" + 0.012*"timor" + 0.011*"storm" + 0.011*"open" + 0.011*"north"

Score: 0.23754115402698517	 
Topic: 0.022*"forc" + 0.019*"concern" + 0.017*"sydney" + 0.017*"howard" + 0.017*"price" + 0.016*"high" + 0.015*"market" + 0.012*"close" + 0.011*"state" + 0.011*"sale"

Score: 0.22000007331371307	 
Topic: 0.064*"polic" + 0.026*"investig" + 0.025*"miss" + 0.019*"death" + 0.019*"continu" + 0.017*"search" + 0.014*"probe" + 0.013*"victim" + 0.012*"trial" + 0.012*"lose"

Score: 0.020012054592370987	 
Topic: 0.031*"fund" + 0.026*"govt" + 0.018*"school" + 0.013*"deal" + 0.013*"protest" + 0.013*"worker" + 0.012*"feder" + 0.011*"plan" + 0.011*"support" + 0.010*"help"

Score: 0.02000676840543747	 
Topic: 0.040*"plan" + 0.040*"council" + 0.036*"water" + 0.021*"govt" + 0.018*"urg" + 0.018*"chang" + 0.015*"power" + 0.014*"back" + 0.013*"resid" + 0.012*"group"

Score: 0.020003862

### It has the highest probability (`x`) to be  part of the topic that we assigned as X, which is the accurate classification. ###

## Performance evaluation by classifying sample document using LDA TF-IDF model##

In [29]:
'''
Check which topic our test document belongs to using the LDA TF-IDF model.
'''
# Our test document is document number 4310
for index, score in sorted(lda_model_tfidf[bow_corpus[document_num]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model_tfidf.print_topic(index, 10)))


Score: 0.49756646156311035	 
Topic: 0.057*"polic" + 0.028*"investig" + 0.019*"death" + 0.017*"support" + 0.014*"victim" + 0.013*"trial" + 0.013*"begin" + 0.012*"bodi" + 0.012*"prompt" + 0.011*"appeal"

Score: 0.3424290716648102	 
Topic: 0.051*"govt" + 0.036*"urg" + 0.023*"fund" + 0.022*"council" + 0.017*"boost" + 0.015*"hous" + 0.015*"reject" + 0.014*"home" + 0.013*"health" + 0.013*"plan"

Score: 0.02000248059630394	 
Topic: 0.034*"crash" + 0.033*"kill" + 0.029*"water" + 0.023*"iraq" + 0.021*"sydney" + 0.018*"power" + 0.015*"die" + 0.013*"attack" + 0.013*"troop" + 0.013*"protest"

Score: 0.020000992342829704	 
Topic: 0.019*"rise" + 0.018*"year" + 0.017*"green" + 0.016*"road" + 0.015*"price" + 0.013*"drought" + 0.012*"region" + 0.011*"seek" + 0.011*"farm" + 0.010*"farmer"

Score: 0.02000097557902336	 
Topic: 0.025*"open" + 0.023*"lead" + 0.016*"win" + 0.015*"england" + 0.015*"feder" + 0.014*"world" + 0.013*"final" + 0.012*"aussi" + 0.010*"victori" + 0.010*"tiger"

Score: 0.019999999552

### It has the highest probability (`x%`) to be  part of the topic that we assigned as X. ###

## Testing model on unseen document ##

In [30]:
unseen_document = "My favorite sports activities are running and swimming."

# Data preprocessing step for the unseen document
bow_vector = dictionary.doc2bow(preprocess(unseen_document))

for index, score in sorted(lda_model[bow_vector], key=lambda tup: -1*tup[1]):
    print("Score: {}\t Topic: {}".format(score, lda_model.print_topic(index, 5)))

Score: 0.22000005841255188	 Topic: 0.031*"fund" + 0.026*"govt" + 0.018*"school" + 0.013*"deal" + 0.013*"protest"
Score: 0.2200000286102295	 Topic: 0.064*"polic" + 0.026*"investig" + 0.025*"miss" + 0.019*"death" + 0.019*"continu"
Score: 0.2199999988079071	 Topic: 0.021*"nation" + 0.021*"hospit" + 0.019*"health" + 0.015*"mayor" + 0.014*"fight"
Score: 0.21999993920326233	 Topic: 0.037*"govt" + 0.027*"urg" + 0.023*"closer" + 0.022*"labor" + 0.022*"claim"
Score: 0.02000000886619091	 Topic: 0.022*"forc" + 0.019*"concern" + 0.017*"sydney" + 0.017*"howard" + 0.017*"price"
Score: 0.019999999552965164	 Topic: 0.029*"crash" + 0.029*"kill" + 0.029*"iraq" + 0.018*"die" + 0.017*"blaze"
Score: 0.019999999552965164	 Topic: 0.015*"indigen" + 0.015*"talk" + 0.015*"warn" + 0.015*"coast" + 0.014*"nuclear"
Score: 0.019999999552965164	 Topic: 0.040*"plan" + 0.040*"council" + 0.036*"water" + 0.021*"govt" + 0.018*"urg"
Score: 0.019999999552965164	 Topic: 0.019*"lead" + 0.019*"rise" + 0.017*"australia" + 0.016

The model correctly classifies the unseen document with 'x'% probability to the X category.